## Importar Librerías

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.gridspec import GridSpec
from scipy import fft
import sys, os
from typing import Dict, Tuple, List, Optional, Callable
import warnings
warnings.filterwarnings('ignore')

sys.path.insert(0, os.path.abspath('../python'))
print("✓ Librerías importadas")

## Definir Tipos de Condiciones de Frontera

In [ ]:
class BoundaryCondition:
    """Clase base para condiciones de frontera"""
    
    def __init__(self, name: str, bc_type: str, location: str):
        self.name = name
        self.bc_type = bc_type
        self.location = location

class DirichletBC(BoundaryCondition):
    def __init__(self, name: str, location: str, value_func: Callable):
        super().__init__(name, 'dirichlet', location)
        self.value_func = value_func
        self.description = f"Dirichlet BC en {location}: u = g(x,t)"

class NeumannBC(BoundaryCondition):
    def __init__(self, name: str, location: str, gradient_func: Callable):
        super().__init__(name, 'neumann', location)
        self.gradient_func = gradient_func
        self.description = f"Neumann BC en {location}: ∂u/∂n = h(x,t)"

class NoSlipBC(BoundaryCondition):
    def __init__(self, name: str = "wall"):
        super().__init__(name, 'noslip', 'wall')
        self.description = "No-slip BC: u = 0 (pared sólida)"

class FreeSlipBC(BoundaryCondition):
    def __init__(self, name: str = "freeslip"):
        super().__init__(name, 'freeslip', 'interface')
        self.description = "Free-slip BC: u·n = 0, τ·u = 0"

print("✓ Clases de BC definidas")

## Configuraciones Físicas

In [ ]:
class LidDrivenCavity:
    """Cavidad con tapa móvil"""
    def __init__(self, lid_velocity: float = 1.0, reynolds: float = 1000):
        self.U0 = lid_velocity
        self.Re = reynolds
        self.nu = 1.0 / reynolds
        self.boundary_conditions = {
            'bottom': NoSlipBC('bottom'),
            'left': NoSlipBC('left'),
            'right': NoSlipBC('right'),
            'top': DirichletBC('top', 'wall', lambda t: self.U0)
        }

class ChannelFlow:
    """Flujo en canal"""
    def __init__(self, u_max: float = 1.0, reynolds: float = 1000):
        self.u_max = u_max
        self.Re = reynolds
        self.nu = 1.0 / reynolds
    
    def poiseuille_profile(self, y: np.ndarray) -> np.ndarray:
        return self.u_max * (1.0 - y**2)

class TurbulentBoundaryLayer:
    """Capa límite turbulenta"""
    def __init__(self, u_infinity: float = 1.0, reynolds: float = 10000):
        self.U_inf = u_infinity
        self.Re = reynolds
        self.nu = 1.0 / reynolds
    
    def blasius_profile(self, eta: np.ndarray) -> np.ndarray:
        return self.U_inf * (1.0 - np.exp(-2.0 * eta))

print("✓ Configuraciones físicas definidas")

## Crear Instancias y Analizar

In [ ]:
# Crear instancias
cavity = LidDrivenCavity(lid_velocity=1.0, reynolds=1000)
channel = ChannelFlow(u_max=1.0, reynolds=1000)
bl = TurbulentBoundaryLayer(u_infinity=1.0, reynolds=10000)

print("\n" + "="*80)
print("ANÁLISIS DE CONDICIONES DE FRONTERA Y PUNTOS CRÍTICOS")
print("="*80)

print(f"\n1️⃣  LID-DRIVEN CAVITY (Re = {cavity.Re})")
print(f"   Viscosidad: ν = {cavity.nu:.6f}")
print(f"   Condiciones de Frontera:")
for name, bc in cavity.boundary_conditions.items():
    print(f"     • {name}: {bc.description}")

print(f"\n2️⃣  CHANNEL FLOW (Re = {channel.Re})")
print(f"   Viscosidad: ν = {channel.nu:.6f}")
print(f"   Perfil Poiseuille en y=0.5: u = {channel.poiseuille_profile(0.5):.4f}")

print(f"\n3️⃣  TURBULENT BOUNDARY LAYER (Re = {bl.Re})")
print(f"   Viscosidad: ν = {bl.nu:.6f}")
print(f"   Escala Kolmogorov: η ~ Re^(-3/4) = {bl.Re**(-0.75):.6f}")
print(f"   Escala viscosa: y⁺ ~ √Re = {np.sqrt(bl.Re):.1f}")

## Visualización: Perfiles de Velocidad

In [ ]:
fig, axes = plt.subplots(1, 3, figsize=(15, 5))

# 1. Cavidad
ax = axes[0]
from matplotlib.patches import Rectangle, Circle
rect = Rectangle((0, 0), 1, 1, fill=False, edgecolor='black', linewidth=2)
ax.add_patch(rect)
ax.arrow(0.1, 1.0, 0.3, 0, head_width=0.03, head_length=0.05, fc='red', ec='red')
ax.text(0.25, 1.08, 'U₀ (Dirichlet)', fontsize=9, color='red', weight='bold')
circle = Circle((0.5, 0.5), 0.2, fill=False, edgecolor='blue', linewidth=2, linestyle='--')
ax.add_patch(circle)
for corner in [(0, 0), (1, 0), (0, 1), (1, 1)]:
    ax.plot(corner[0], corner[1], 'go', markersize=8)
ax.set_xlim(-0.1, 1.1)
ax.set_ylim(-0.1, 1.1)
ax.set_aspect('equal')
ax.set_title('Lid-Driven Cavity\n3 Fuerzas: ∇p + ν∇²u + (u·∇)u', fontsize=10, weight='bold')
ax.set_xlabel('x')
ax.set_ylabel('y')
ax.grid(True, alpha=0.3)

# 2. Canal
ax = axes[1]
y = np.linspace(-1, 1, 100)
u_poiseuille = channel.poiseuille_profile(y)
ax.fill_betweenx(y, 0, u_poiseuille, alpha=0.3, color='blue')
ax.plot(u_poiseuille, y, 'b-', linewidth=2)
ax.axhline(y=1, color='red', linestyle='-', linewidth=2)
ax.axhline(y=-1, color='red', linestyle='-', linewidth=2)
ax.arrow(0.3, 0, 0.2, 0, head_width=0.1, head_length=0.05, fc='green', ec='green')
ax.text(0.4, 0.2, '∇p', fontsize=10, color='green', weight='bold')
ax.set_title('Channel Flow\nBalance: ∇p + ν∇²u = 0', fontsize=10, weight='bold')
ax.set_xlabel('Velocidad u(y)')
ax.set_ylabel('Posición y')
ax.set_xlim(-0.1, 1.2)
ax.grid(True, alpha=0.3)

# 3. Capa límite
ax = axes[2]
y_bl = np.linspace(0, 1, 100)
u_bl = bl.blasius_profile(y_bl)
ax.fill_between(u_bl, y_bl, alpha=0.3, color='purple')
ax.plot(u_bl, y_bl, 'purple', linewidth=2, label='Blasius')
ax.axhline(y=0, color='black', linewidth=3)
ax.text(0.5, 0.02, 'Pared (u=0, τ_max)', fontsize=8, ha='center')
ax.fill_between(u_bl[y_bl < 0.1], 0, y_bl[y_bl < 0.1], alpha=0.5, color='red', label='Subcapa viscosa')
ax.set_title('Turbulent Boundary Layer\nPared + Núcleo turbulento', fontsize=10, weight='bold')
ax.set_xlabel('Velocidad u')
ax.set_ylabel('Posición y')
ax.legend(loc='lower right')
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('../boundary_conditions_profiles.png', dpi=150, bbox_inches='tight')
plt.show()

print("\n✓ Visualización guardada: boundary_conditions_profiles.png")

## Tabla Comparativa de Escalas

In [ ]:
import pandas as pd

# Tabla de escalas y puntos críticos
escalas_df = pd.DataFrame({
    'Configuración': ['Cavidad', 'Canal', 'Capa Límite'],
    'Reynolds': [1000, 1000, 10000],
    'Capa Límite (δ)': [f'{1000**(-0.25):.4f}', f'{(1000**0.5 * np.pi)**(-0.5):.4f}', 'δ(x) variable'],
    'Puntos Críticos': ['4 esquinas + centro', 'Entrada', 'Pared + Núcleo'],
    'Fuerza Dominante': ['Advección', 'Presión-Viscosidad', 'Turbulencia'],
    'Complejidad': ['ALTA', 'MEDIA', 'MÁXIMA']
})

print("\n" + "="*120)
print("TABLA: ESCALAS Y PUNTOS CRÍTICOS")
print("="*120)
print(escaolas_df.to_string(index=False))
print("="*120)

# Tabla de tipos de BC
bc_df = pd.DataFrame({
    'Tipo BC': ['Dirichlet', 'Neumann', 'No-slip', 'Free-slip', 'Periódica'],
    'Símbolo': ['u = g', '∂u/∂n = h', 'u = 0', 'u·n = 0', 'u(x)=u(x+L)'],
    'Significado': ['Velocidad prescrita', 'Gradiente prescrito', 'Adherencia a pared', 'Sin fricción', 'Periodicidad'],
    'Ejemplo': ['Inlet', 'Outlet', 'Cavidad', 'Interfaz', 'DNS'],
})

print("\n" + "="*100)
print("TABLA: TIPOS DE CONDICIONES DE FRONTERA")
print("="*100)
print(bc_df.to_string(index=False))
print("="*100)

## Análisis de Puntos Críticos de Interacción de Fuerzas

In [ ]:
print("\n" + "="*80)
print("ANÁLISIS: PUNTOS CRÍTICOS DE INTERACCIÓN DE FUERZAS")
print("="*80)

print("\n1️⃣  CAVIDAD CON TAPA MÓVIL")
print("-" * 80)
print("\n▸ ESQUINAS (Puntos críticos de vorticidad concentrada)")
print("  - Fuerzas: Viscosidad (esfuerzo cortante) + Presión")
print("  - Escala de longitud: δ ~ Re^(-0.25) ≈ 0.1778")
print("  - Efecto: Micro-vórtices secundarios")
print("  - Característica: Re-dependiente (más intensos con Re↑)")

print("\n▸ CENTRO DEL VÓRTICE PRINCIPAL")
print("  - Fuerzas: Advección (u·∇)u ↔ Difusión ν∇²u")
print("  - Balance: Dinámicas cuasi-equilibrio")
print("  - Fenómeno: Posibles oscilaciones periódicas o caóticas")
print("  - Sensibilidad: Alta a perturbaciones numéricas")

print("\n2️⃣  FLUJO EN CANAL")
print("-" * 80)
print("\n▸ REGIÓN DE ENTRADA")
print(f"  - Longitud de desarrollo: L_e ~ 0.05·Re·D ≈ 50·D")
print(f"  - Fuerzas: ∇p (impulsa) ↔ ν∇²u (resiste)")
print("  - Punto crítico: Donde se establece el perfil")
print("  - Fenómeno: Transición entrada → flujo desarrollado")

print("\n▸ FLUJO DESARROLLADO")
print("  - Balance de fuerzas: ∂u/∂t = 0, ∇·u = 0")
print("  - Ecuación: ∇p + ν∇²u = 0 (estado permanente)")
print("  - Vorticidad: ω_z = ∂v/∂x - ∂u/∂y (solo transversal)")
print("  - Disipación: ε = ν|∇u|² uniforme en y")

print("\n3️⃣  CAPA LÍMITE TURBULENTA")
print("-" * 80)
print("\n▸ SUBCAPA VISCOSA (y⁺ < 5)")
print(f"  - Escala: Dominancia viscosidad >> advección")
print(f"  - Perfil: u⁺ = y⁺ (lineal)")
print(f"  - Esfuerzo pared: τ_wall = μ(∂u/∂y)|_wall = MÁXIMO")
print(f"  - Rol físico: GENERACIÓN MÁXIMA de vorticidad")

print("\n▸ BUFFER LAYER (5 < y⁺ < 30)")
print("  - Transición: viscosidad ↔ advección (balance delicado)")
print("  - Perfil: u⁺ = (1/κ)·ln(y⁺) + C (von Kármán)")
print("  - Fenómeno: Bursts y sweep events (eyecciones/barridos)")
print("  - Importancia: Interfaz viscosidad-turbulencia")

print("\n▸ NÚCLEO TURBULENTO (y⁺ > 30)")
print(f"  - Dinámicas: Turbulencia desarrollada")
print(f"  - Balance de fuerzas: PRODUCCIÓN = DISIPACIÓN")
print(f"  - Estructuras: Streaks longitudinales, vórtices")
print(f"  - Escala más pequeña (Kolmogorov): η ~ Re^(-3/4) ≈ {10000**(-0.75):.6f}")
print(f"  - Rango de escalas: δ/η ~ Re^(3/4) ≈ {10000**(0.75):.0f} (enorme)")

## Conclusiones y Perspectivas

In [ ]:
print("\n" + "="*80)
print("CONCLUSIONES Y PERSPECTIVAS FUTURAS")
print("="*80)

print("""
✅ HALLAZGOS PRINCIPALES:

1. CAVIDAD CON TAPA MÓVIL:
   - Sistema con múltiples puntos críticos
   - Interacción compleja de 3 fuerzas (presión, viscosidad, advección)
   - Ideal para validación de métodos numéricos
   - Newton-Bernstein: Refinar en esquinas

2. FLUJO EN CANAL:
   - Balance de fuerzas relativamente simple y predecible
   - Soluciones analíticas disponibles (Poiseuille)
   - Punto crítico principal: región de entrada
   - Newton-Bernstein: Refinar en entrada

3. CAPA LÍMITE TURBULENTA:
   - Máxima complejidad: turbulencia + múltiples escalas
   - Rango de escalas: Re^(3/4) (vastísimo para Re grande)
   - Puntos críticos bien definidos (pared, buffer, núcleo)
   - Newton-Bernstein: Refinar en subcapa viscosa

📋 APLICACIONES FUTURAS:

1. Bifurcaciones: Estudiar transiciones de estabilidad vs Re
2. Control de flujo: Encontrar condiciones de frontera óptimas
3. Machine Learning: Predicción de flujos con BC dados
4. Análisis adaptativo: Newton-Bernstein en puntos críticos
5. Optimización topológica: Diseño de geometrías óptimas

🔬 IMPLICACIONES PARA NAVIER-STOKES 3D:

- Las BC determinan la estructura de la solución
- Puntos críticos localizan donde pueden ocurrir singularidades
- Refinamiento adaptativo es clave para resolver dinámicas complejas
- Métodos espectrales requieren tratamiento cuidadoso de BC
""")

print("="*80)
print("\n✓ Análisis completado")